In [1]:
import sys
sys.path.append(r'C:\Users\HP\anaconda3\envs\timeseries\Lib\site-packages')

In [27]:
import pandas as pd
from prophet import Prophet

# Load the entire historical dataset
df_prophet = pd.read_csv('test_data.csv')

# Convert 'Year' and 'Month' columns to datetime
df_prophet['Date'] = pd.to_datetime(df_prophet['Year'].astype(str) + '-' + df_prophet['Month'], format='%Y-%B')

# Set 'Date' column as index
df_prophet.index = df_prophet['Date']

# Drop 'Year' and 'Month' columns if not needed anymore
df_prophet.drop(columns=['Year', 'Month'], inplace=True)

# Rename columns to 'ds' and 'y'
df_prophet.rename(columns={'Date':'ds','QTY': 'y'}, inplace=True)

# Initialize Prophet model
m = Prophet(
    changepoint_prior_scale=0.08, 
    seasonality_prior_scale=10.0,
    seasonality_mode='multiplicative',  # Adjust seasonality mode if needed
    yearly_seasonality=True
)


# Fit the model with all available historical data
m.fit(df_prophet)

def forecast_api(start_date, end_date, forecast_horizon):
    # Generate future dates for the forecast horizon within the specified date range
    future = pd.date_range(start=start_date, periods=forecast_horizon, freq='MS')

    # Convert future dates to dataframe
    future_df = pd.DataFrame({'ds': future})
    future_df['ds'] = future_df['ds'] + pd.offsets.MonthBegin(1)


    # Make forecasts
    
    forecast = m.predict(future_df)

    # Return forecasts for the specified forecast horizon
    return forecast.tail(forecast_horizon)

# Example usage of the API
start_date = '2023-01-01'
end_date = '2023-12-31'
forecast_horizon = 12 # Forecast for the next 12 months

forecast = forecast_api(start_date, end_date, forecast_horizon)


INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:finish chain 1


In [28]:
forecast.tail(forecast_horizon)


,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yearly,yearly_lower,yearly_upper,additive_terms,additive_terms_lower,additive_terms_upper,yhat
0,2023-02-01,29048.984966,39480.371708,39515.615863,29048.984966,29048.984966,0.359713,0.359713,0.359713,0.359713,0.359713,0.359713,0.0,0.0,0.0,39498.289387
1,2023-03-01,28833.684727,2619.999722,2656.524535,28833.684727,28833.684727,-0.908499,-0.908499,-0.908499,-0.908499,-0.908499,-0.908499,0.0,0.0,0.0,2638.323212
2,2023-04-01,28595.316606,58927.720806,58964.157908,28595.316606,28595.316606,1.061417,1.061417,1.061417,1.061417,1.061417,1.061417,0.0,0.0,0.0,58946.874954
3,2023-05-01,28364.637778,32406.907569,32441.306589,28364.637778,28364.637778,0.143125,0.143125,0.143125,0.143125,0.143125,0.143125,0.0,0.0,0.0,32424.330005
4,2023-06-01,28126.269656,46052.789535,46089.556531,28126.269656,28126.269656,0.637999,0.637999,0.637999,0.637999,0.637999,0.637999,0.0,0.0,0.0,46070.791295
5,2023-07-01,27895.590829,57909.789744,57947.000795,27895.590829,27895.590829,1.076604,1.076604,1.076604,1.076604,1.076604,1.076604,0.0,0.0,0.0,57928.099233
6,2023-08-01,27657.222707,31037.021392,31073.691871,27657.222707,27657.222707,0.122864,0.122864,0.122864,0.122864,0.122864,0.122864,0.0,0.0,0.0,31055.288271
7,2023-09-01,27418.854586,21468.847764,21506.898533,27418.854586,27418.854586,-0.216295,-0.216295,-0.216295,-0.216295,-0.216295,-0.216295,0.0,0.0,0.0,21488.293614
8,2023-10-01,27188.175758,7497.659037,7537.636938,27188.175758,27188.175758,-0.723516,-0.723516,-0.723516,-0.723516,-0.723516,-0.723516,0.0,0.0,0.0,7517.098438
9,2023-11-01,26949.807637,18476.419849,18512.870120,26949.807637,26949.807637,-0.313776,-0.313776,-0.313776,-0.313776,-0.313776,-0.313776,0.0,0.0,0.0,18493.600454
